In [2]:
import pandas as pd
from xgboost import XGBRegressor
from datetime import datetime
import joblib
import numpy as np

In [11]:
model = XGBRegressor()
model.load_model("../../checkpoints/xgb_model.json")

fs = pd.read_parquet("../../data/feature_store/tmp_feature_store.parquet")
df = pd.read_parquet("../../data/processed_data/train.parquet").drop(columns=["log_view_count"])

ohe_category = joblib.load("../../artifacts/ohe_encoder_category.joblib")

In [88]:
# Explicit:
channel = "CGP Grey"
category = "Education"

# Implicit:
year = 2023
month = 3
day = 21

today = pd.Series(pd.to_datetime(datetime.now().date()))

In [89]:
today.dt.year
today.dt.month
today.dt.day_of_week

0    5
dtype: int64

In [90]:
fs_channel = (
    fs
    .drop_duplicates(subset=['channelTitle'])
    [["channelTitle", "channel_trending_videos_last_7D", "log_channel_avg_n_views_7D", "log_channel_avg_n_comments_7D", "log_channel_view_like_ratio_7D"]]
)
fs_category = (
    fs
    .drop_duplicates(subset=['category'])
    [["category", "category_trending_videos_last_7D", "category_avg_n_views_7D", "category_avg_n_comments_7D", "category_view_like_ratio_7D"]]
)

In [91]:
category_encoded = pd.DataFrame(
    ohe_category.transform(np.array(category).reshape(-1, 1)).toarray(),
    columns=ohe_category.get_feature_names_out(),
    )
category_encoded.columns = [
    (f"category_" + col.split("_")[1]).replace(" ", "") for col in category_encoded.columns
    ]

In [92]:
request = pd.concat([
    fs_channel[fs_channel['channelTitle'] == channel].drop(columns=["channelTitle"]).reset_index(drop=True),
    fs_category[fs_category['category'] == category].drop(columns=["category"]).reset_index(drop=True),
    pd.DataFrame(today.dt.year, columns=["year"]),
    pd.DataFrame(today.dt.month, columns=["month"]),
    pd.DataFrame((today.dt.day_of_week <= 4).astype(int), columns=["is_weekday"]),
    category_encoded,
    ], axis=1
)

request = request[list(df.columns)]

In [98]:
prediction = model.predict(request).item()
prediction = np.exp(prediction)
print(prediction)

2788865.709626342
